In [1]:
from config import model_api

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import requests

In [89]:
ticker = "AYRO"
incomeAnnual_url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={model_api}"
# get income statement data
incomeQuarter_url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={model_api}&period=quarter&limit=100"
# get cash flow statement data
cashFlowQuarter_url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?apikey={model_api}&period=quarter&limit=100"
# key stats investors look for
keyStats_url = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={model_api}"
# income statement growth numbers
incomeGrowthQuarter_url = f"https://financialmodelingprep.com/api/v3/income-statement-growth/{ticker}?limit=100&period=quarter&apikey={model_api}"

In [90]:
income_json = requests.get(incomeQuarter_url).json()
cashFlow_json = requests.get(cashFlowQuarter_url).json()


In [37]:
keyStats_json = requests.get(keyStats_url).json()

In [35]:
keyStats_url

[{'symbol': 'AYRO',
  'price': 8.64,
  'beta': 2.61234,
  'volAvg': 8851822,
  'mktCap': 247728688,
  'lastDiv': 0.0,
  'range': '1.8-11.5',
  'changes': -0.28,
  'companyName': 'AYRO Inc',
  'currency': 'USD',
  'cik': None,
  'isin': 'US0547481087',
  'cusip': '054748108',
  'exchange': 'Nasdaq Global Select',
  'exchangeShortName': 'NASDAQ',
  'industry': 'Auto Manufacturers',
  'website': 'https://ayro.com/',
  'description': 'AYRO, Inc. (AYRO), formerly DropCar, Inc., is a designer and manufacturer of purpose-built, automotive-grade electric vehicles (EV). The Company’s EV models include AYRO 311 and Club Car 411. AYRO 311 is a three-wheeled vehicle with inline seating and four-door access for professional and personal use. The Company offers its AYRO 311 with multiple configurations including fully enclosed two-seater, half doors and one-seater with the cargo area. The Club Car 411 is a compact all-electric vehicle suitable for low-speed logistics and cargo services. The Company 

In [42]:
# request for income growth data
incomeGrowth_json = requests.get(incomeGrowthQuarter_url).json()

In [96]:
income_json

[{'date': '2020-09-30',
  'symbol': 'AYRO',
  'reportedCurrency': 'USD',
  'fillingDate': '2020-11-06',
  'acceptedDate': '2020-11-06 08:00:37',
  'period': 'Q3',
  'revenue': 388654.0,
  'costOfRevenue': 326671.0,
  'grossProfit': 61983.0,
  'grossProfitRatio': 0.159481183777859,
  'researchAndDevelopmentExpenses': 664145.0,
  'generalAndAdministrativeExpenses': 1786898.0,
  'sellingAndMarketingExpenses': 304880.0,
  'otherExpenses': 17503.0,
  'operatingExpenses': 2451043.0,
  'costAndExpenses': 2777714.0,
  'interestExpense': 95469.0,
  'depreciationAndAmortization': 146177.0,
  'ebitda': -2225380.0,
  'ebitdaratio': -5.72586413622399,
  'operatingIncome': -2389060.0,
  'operatingIncomeRatio': -6.8974614953146,
  'totalOtherIncomeExpensesNet': -196197.0,
  'incomeBeforeTax': -2680726.0,
  'incomeBeforeTaxRatio': -6.8974614953146,
  'incomeTaxExpense': 0.0,
  'netIncome': -2680726.0,
  'netIncomeRatio': -6.8974614953146,
  'eps': -0.13,
  'epsdiluted': -0.13,
  'weightedAverageShsOut

In [101]:
# create a funciton that outputs all financial statement information
def createIncomeStatement(incomeStatement_file, incomeGrowth_file):
    for i in range(len(incomeStatement_file)):
        # define the rows for the DataFrame
        rows = ["Revenue",
                "Revenue Growth",
                "Cost of Revenue",
                "Gross Profit",
                'Operating Margin', 
                "Operating Income/Loss", 
                "Operating Income/Loss Growth",                  
                "Net Income"]

        # determine revenue
        revenue = incomeStatement_file[i]['revenue'] / 1000
        # determine revenue growth
        try:
            revenuePreviousQuarter = incomeStatement_file[i+1]['revenue'] / 1000
            revenueGrowth = (revenue - revenuePreviousQuarter) / revenuePreviousQuarter
        except IndexError:
            revenueGrowth = np.nan
        # determine cost of revenue
        costRevenue = incomeStatement_file[i]['costOfRevenue'] / 1000
        # determine gross profit
        gross_profit = incomeStatement_file[i]['grossProfit'] / 1000
        # determine operating expenses
        operating_expenses = incomeStatement_file[i]['operatingExpenses'] / 1000
        # determine net income
        net_income = incomeStatement_file[i]['netIncome'] / 1000
        # calculate operating income
        operating_income = gross_profit - operating_expenses
        # operating income growth
        try:
            grossProfitPreviousQuarter = incomeStatement_file[i+1]['grossProfit'] / 1000
            operatingExpensesPreviousQuarter = incomeStatement_file[i+1]['operatingExpenses'] / 1000
            operatingIncomePreviousQuarter = grossProfitPreviousQuarter - operatingExpensesPreviousQuarter
            operating_income_growth = (operating_income - operatingIncomePreviousQuarter) / operatingIncomePreviousQuarter
        except IndexError:
            operating_income_growth = np.nan
        # calculate operating margin
        operating_margin = operating_income / revenue
        
        # create a dataframe on the first iteration and add new columns on the rest 
        if i == 0:
            df = pd.DataFrame({incomeStatement_file[i]['date']: [revenue,
                                                                 revenueGrowth,
                                                                 costRevenue,
                                                                 gross_profit,
                                                               operating_margin, 
                                                               operating_income,
                                                               operating_income_growth,                                                               
                                                               net_income]}, 
                                     index=rows)
        else:
            df[incomeStatement_file[i]['date']] = [revenue,
                                                   revenueGrowth,
                                                   costRevenue,
                                                   gross_profit,
                                                   operating_margin, 
                                                   operating_income,
                                                   operating_income_growth,                                                   
                                                   net_income]
    # display company ticker as index title
    df.index.name = incomeStatement_file[0]['symbol']
    
    
    return df

In [102]:
createIncomeStatement(income_json, incomeGrowth_json)

,2020-09-30,2020-06-30,2020-03-31,2019-12-31,2019-09-30
AYRO,,,,,
Revenue,388.654000,285.927000,146.820000,457.974000,265.481000
Revenue Growth,0.359277,0.947466,-0.679414,0.725073,NaN
Cost of Revenue,326.671000,205.637000,113.160000,417.232000,202.029000
Gross Profit,61.983000,80.290000,33.660000,40.742000,63.452000
Operating Margin,-6.147010,-3.686462,-11.507560,-5.319992,-7.826846
Operating Income/Loss,-2389.060000,-1054.059000,-1689.540000,-2436.418000,-2077.879000
Operating Income/Loss Growth,1.266533,-0.376127,-0.306548,0.172550,NaN
Net Income,-2680.726000,-1530.857000,-1795.150000,-4902.380000,-2141.840000


In [47]:
pd.DataFrame({income_json[0]['date']: [revenue,
                                       operating_margin, 
                                       operating_income,
                                       operating_income_growth,
                                       gross_profit,
                                       net_income]}, 
             index=rows)

,2020-09-30
Revenue,388.654000
Operating Margin,-6.147010
Operating Income/Loss,-2389.060000
Operating Income Growth,1696.643681
Gross Profit,61.983000
Net Income,-2680.726000


In [22]:
cashFlow_json

[{'date': '2020-09-30',
  'symbol': 'AYRO',
  'reportedCurrency': 'USD',
  'fillingDate': '2020-11-06',
  'acceptedDate': '2020-11-06 08:00:37',
  'period': 'Q3',
  'netIncome': -2680726.0,
  'depreciationAndAmortization': 146177.0,
  'deferredIncomeTax': 0.0,
  'stockBasedCompensation': 167767.0,
  'changeInWorkingCapital': -1742486.0,
  'accountsReceivables': -105307.0,
  'inventory': -466128.0,
  'accountsPayables': 226716.0,
  'otherWorkingCapital': 58610.0,
  'otherNonCashItems': 66659.0,
  'netCashProvidedByOperatingActivities': -3824572.0,
  'investmentsInPropertyPlantAndEquipment': 0.0,
  'acquisitionsNet': 0.0,
  'purchasesOfInvestments': 0.0,
  'salesMaturitiesOfInvestments': 0.0,
  'otherInvestingActivites': 0.0,
  'netCashUsedForInvestingActivites': 0.0,
  'debtRepayment': -639483.0,
  'commonStockIssued': 22335003,
  'commonStockRepurchased': 0.0,
  'dividendsPaid': 0.0,
  'otherFinancingActivites': 0.0,
  'netCashUsedProvidedByFinancingActivities': 0.0,
  'effectOfForexCh

In [26]:
## CASH FLOW STATEMENT
cashRows = ["cash from operating activities",
           "cash used for investing activities",
           "cash used provided by financing activities",
           "acquisitions",
           "purchase of investments",
           "sales maturities of investments",
           "net change in cash",
           "free cash flow",
           "common stock repurchased"]
cashOperatingActivities = cashFlow_json[0]['netCashProvidedByOperatingActivities']
cashInvestingActivities = cashFlow_json[0]['netCashUsedForInvestingActivites']
cashFinancingActivities = cashFlow_json[0]['netCashUsedProvidedByFinancingActivities']
acquisitions = cashFlow_json[0]['acquisitionsNet']
purchaseInvestments = cashFlow_json[0]['purchasesOfInvestments']
investmentMaturities = cashFlow_json[0]['salesMaturitiesOfInvestments']
changeCash = cashFlow_json[0]['netChangeInCash']
freeCashFlow = cashFlow_json[0]['freeCashFlow']
stockRepurchase = cashFlow_json[0]['commonStockRepurchased']

In [29]:
pd.DataFrame({cashFlow_json[0]['date']: [cashOperatingActivities,
                                        cashInvestingActivities,
                                        cashFinancingActivities,
                                        acquisitions,
                                        purchaseInvestments,
                                        investmentMaturities,
                                        changeCash,
                                        freeCashFlow,
                                        stockRepurchase]}, 
             index=cashRows)

,2020-09-30
cash from operating activities,-3824572.0
cash used for investing activities,0.0
cash used provided by financing activities,0.0
acquisitions,0.0
purchase of investments,0.0
sales maturities of investments,0.0
net change in cash,19998718.0
free cash flow,-4164991.0
common stock repurchased,0.0


In [16]:
test = requests.get(keyStats_url)

In [22]:
test2 = requests.get(incomeStatement_url)

In [25]:
test2.json()[0]

{'date': '2020-09-26',
 'symbol': 'AAPL',
 'reportedCurrency': 'USD',
 'fillingDate': '2020-10-30',
 'acceptedDate': '2020-10-29 18:06:25',
 'period': 'FY',
 'revenue': 274515000000,
 'costOfRevenue': 169559000000,
 'grossProfit': 104956000000,
 'grossProfitRatio': 0.382332477278109,
 'researchAndDevelopmentExpenses': 18752000000,
 'generalAndAdministrativeExpenses': 19916000000,
 'sellingAndMarketingExpenses': 0.0,
 'otherExpenses': -87000000,
 'operatingExpenses': 38668000000,
 'costAndExpenses': 208227000000,
 'interestExpense': 2873000000,
 'depreciationAndAmortization': 11056000000,
 'ebitda': 81020000000,
 'ebitdaratio': 0.295138699160337,
 'operatingIncome': 66288000000,
 'operatingIncomeRatio': 0.244398302460703,
 'totalOtherIncomeExpensesNet': -87000000,
 'incomeBeforeTax': 67091000000,
 'incomeBeforeTaxRatio': 0.244398302460703,
 'incomeTaxExpense': 9680000000,
 'netIncome': 57411000000,
 'netIncomeRatio': 0.209136112780722,
 'eps': 3.36,
 'epsdiluted': 3.36,
 'weightedAverag

In [26]:
test2.json()[1]

{'date': '2019-09-28',
 'symbol': 'AAPL',
 'reportedCurrency': 'USD',
 'fillingDate': '2019-10-31 00:00:00',
 'acceptedDate': '2019-10-30 18:12:36',
 'period': 'FY',
 'revenue': 260174000000,
 'costOfRevenue': 161782000000,
 'grossProfit': 98392000000,
 'grossProfitRatio': 0.3781776810903472,
 'researchAndDevelopmentExpenses': 16217000000,
 'generalAndAdministrativeExpenses': 18245000000,
 'sellingAndMarketingExpenses': 0.0,
 'otherExpenses': 422000000,
 'operatingExpenses': 34462000000,
 'costAndExpenses': 196244000000,
 'interestExpense': 3576000000,
 'depreciationAndAmortization': 12547000000,
 'ebitda': 81860000000,
 'ebitdaratio': 0.3146355900282119,
 'operatingIncome': 63930000000,
 'operatingIncomeRatio': 0.2526655238417367,
 'totalOtherIncomeExpensesNet': 422000000,
 'incomeBeforeTax': 65737000000,
 'incomeBeforeTaxRatio': 0.2526655238417367,
 'incomeTaxExpense': 10481000000,
 'netIncome': 55256000000,
 'netIncomeRatio': 0.21238094505984456,
 'eps': 2.9925,
 'epsdiluted': 2.972

In [27]:
test2.json()[2]

{'date': '2018-09-29',
 'symbol': 'AAPL',
 'reportedCurrency': 'USD',
 'fillingDate': '2018-11-05 00:00:00',
 'acceptedDate': '2018-11-05 08:01:40',
 'period': 'FY',
 'revenue': 265595000000,
 'costOfRevenue': 163756000000,
 'grossProfit': 101839000000,
 'grossProfitRatio': 0.38343718820007905,
 'researchAndDevelopmentExpenses': 14236000000,
 'generalAndAdministrativeExpenses': 16705000000,
 'sellingAndMarketingExpenses': 0.0,
 'otherExpenses': -441000000,
 'operatingExpenses': 30941000000,
 'costAndExpenses': 194697000000,
 'interestExpense': 3240000000,
 'depreciationAndAmortization': 10903000000,
 'ebitda': 87046000000,
 'ebitdaratio': 0.327739603531693,
 'operatingIncome': 70898000000,
 'operatingIncomeRatio': 0.27448935409175623,
 'totalOtherIncomeExpensesNet': -441000000,
 'incomeBeforeTax': 72903000000,
 'incomeBeforeTaxRatio': 0.27448935409175623,
 'incomeTaxExpense': 13372000000,
 'netIncome': 59531000000,
 'netIncomeRatio': 0.22414202074587247,
 'eps': 3.0025,
 'epsdiluted': 

In [29]:
test2.json()[4]

{'date': '2016-09-24',
 'symbol': 'AAPL',
 'reportedCurrency': 'USD',
 'fillingDate': '2016-10-26 00:00:00',
 'acceptedDate': '2016-10-26 16:42:16',
 'period': 'FY',
 'revenue': 215639000000,
 'costOfRevenue': 131376000000,
 'grossProfit': 84263000000,
 'grossProfitRatio': 0.3907595564809705,
 'researchAndDevelopmentExpenses': 10045000000,
 'generalAndAdministrativeExpenses': 14194000000,
 'sellingAndMarketingExpenses': 0.0,
 'otherExpenses': -1195000000,
 'operatingExpenses': 24239000000,
 'costAndExpenses': 155615000000,
 'interestExpense': 1456000000,
 'depreciationAndAmortization': 10505000000,
 'ebitda': 73333000000,
 'ebitdaratio': 0.34007299236223504,
 'operatingIncome': 60024000000,
 'operatingIncomeRatio': 0.284605289395703,
 'totalOtherIncomeExpensesNet': -1195000000,
 'incomeBeforeTax': 61372000000,
 'incomeBeforeTaxRatio': 0.284605289395703,
 'incomeTaxExpense': 15685000000,
 'netIncome': 45687000000,
 'netIncomeRatio': 0.211867983064288,
 'eps': 2.0875,
 'epsdiluted': 2.07

In [32]:
test2.json()[0]['revenue']

274515000000

In [ ]:
revenue = test2